# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import sys

#scikit
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# magic word for producing visualizations in notebook
%matplotlib inline

KeyboardInterrupt: 

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data

dtype={'CAMEO_DEUG_2015': 'object',
       'CAMEO_INTL_2015': 'object',
       'CJT_GESAMTTYP': 'float64',
       'CJT_KATALOGNUTZER': 'float64',
       'CJT_TYP_1': 'float64',
       'CJT_TYP_2': 'float64',
       'CJT_TYP_3': 'float64',
       'CJT_TYP_4': 'float64',
       'CJT_TYP_5': 'float64',
       'CJT_TYP_6': 'float64',
       'GFK_URLAUBERTYP': 'float64',
       'HH_EINKOMMEN_SCORE': 'float64',
       'LP_FAMILIE_FEIN': 'float64',
       'LP_FAMILIE_GROB': 'float64',
       'LP_LEBENSPHASE_FEIN': 'float64',
       'LP_LEBENSPHASE_GROB': 'float64',
       'LP_STATUS_FEIN': 'float64',
       'LP_STATUS_GROB': 'float64',
       'ONLINE_AFFINITAET': 'float64',
       'RETOURTYP_BK_S': 'float64',
       'RT_KEIN_ANREIZ': 'float64',
       'RT_SCHNAEPPCHEN': 'float64',
       'RT_UEBERGROESSE': 'float64',
       'AKT_DAT_KL': 'float64',
       'ALTER_HH': 'float64',
       'ANZ_KINDER': 'float64',
       'ANZ_PERSONEN': 'float64',
       'ANZ_TITEL': 'float64',
       'D19_BANKEN_ONLINE_QUOTE_12': 'float64',
       'D19_GESAMT_ONLINE_QUOTE_12': 'float64',
       'D19_KONSUMTYP': 'float64',
       'D19_LOTTO': 'float64',
       'D19_SOZIALES': 'float64',
       'D19_TELKO_ONLINE_QUOTE_12': 'float64',
       'D19_VERSAND_ONLINE_QUOTE_12': 'float64',
       'D19_VERSI_ONLINE_QUOTE_12': 'float64',
       'EINGEZOGENAM_HH_JAHR': 'float64',
       'KONSUMNAEHE': 'float64',
       'SOHO_KZ': 'float64',
       'TITEL_KZ': 'float64',
       'UNGLEICHENN_FLAG': 'float64',
       'VHA': 'float64',
       'VK_DHT4A': 'float64',
       'VK_DISTANZ': 'float64',
       'VK_ZG11': 'float64',
       'WOHNDAUER_2008': 'float64'}

azdias = dd.read_csv('data/Udacity_AZDIAS_052018.csv', sep=';', dtype = dtype)
customers = dd.read_csv('data/Udacity_CUSTOMERS_052018.csv', sep=';', dtype = dtype)

### `0.1` Data Understanding

In [ ]:
azdias.head()

In [ ]:
azdias.describe().compute()

#### `0.1.1` Convert float data to int

Convert float data to int if possible

In [ ]:
azdias.dtypes.value_counts()

In [ ]:
for col in azdias.select_dtypes('float').columns:
    azdias[col] = azdias[col].astype('int')

In [ ]:
azdias.dtypes.value_counts()

In [ ]:
customers.dtypes.value_counts()

In [ ]:
for col in customers.select_dtypes('float').columns:
    customers[col] = customers[col].astype('int')

In [ ]:
customers.dtypes.value_counts()

#### `0.1.2` Removing unknown values

**Strategy**
* Read the Excel Attributes
* Eliminate all Unkown value and replace by nan

In [ ]:
azdias_attr = pd.read_excel("DIAS Attributes - Values 2017.xlsx", header = 1, usecols = "B:E")

In [ ]:
azdias_attr.head(15)

Each row with a value in `Attribute` is the first element of a group, so let's fill in the empty values through a front fill.

First, let's check if there is any empty data in `Value`

In [ ]:
azdias_attr.Value.hasnans

There is no empty values in `Value`, so let's proceed with the front fill.

In [ ]:
azdias_attr = azdias_attr.ffill()

In [ ]:
azdias_attr.head()

In [ ]:
pat = re.compile('[Uu]nknown')

In [ ]:
unknows_id = [(re.search(pat, txt) is not None )for txt in azdias_attr.Meaning]

In [ ]:
azdias_attr[unknows_id]

In [ ]:
cols = np.intersect1d(azdias.columns.values, azdias_attr[unknows_id]['Attribute'].values)

In [ ]:
print(f"{len(cols)} cols with Unkown values")

In [ ]:
replacement = dict()
for col in cols:
    values = azdias_attr[unknows_id].query('Attribute == @col').Value.values
    if isinstance(values[0], str):
        values = values[0].split(',')
        values = map(int, values)
    aux = dict()
    for i in values:
        aux[i] = np.nan
    replacement[col] = aux

In [ ]:
azdias = azdias.replace(replacement)

In [ ]:
customers = customers.replace(replacement)

#### `0.1.3` Convert some data types to the appropriate data types

In [ ]:
azdias.compute()

In [ ]:
azdias_nuniques = azdias.nunique()

In [ ]:
azdias_nuniques.compute()

In [ ]:
azdias[azdias_nuniques[azdias_nuniques > 99].index.values]

`EINGEFUEGT_AM` shoud be date format instead of object

In [ ]:
azdias['EINGEFUEGT_AM'] = pd.to_datetime(azdias['EINGEFUEGT_AM'])

In [ ]:
azdias['EINGEFUEGT_AM'].dtypes

In [ ]:
azdias.dtypes.value_counts()

In [ ]:
customers['EINGEFUEGT_AM'] = pd.to_datetime(customers['EINGEFUEGT_AM'])

#### `0.1.4` Exploratory analysis

Let's take a look the extra variables in customers

In [ ]:
cust_cols = np.setdiff1d(customers.columns, azdias.columns)

In [ ]:
customers[cust_cols].nunique()

In [ ]:
customers_groups =  customers[cust_cols].groupby(['CUSTOMER_GROUP', 'PRODUCT_GROUP']).agg(['min', 'max', 'mean', 'std', 'sum', 'count'])


In [ ]:
customers_groups

In [ ]:
labels = customers_groups.loc['MULTI_BUYER'].index.values

multi_product = customers_groups.loc['MULTI_BUYER']['ONLINE_PURCHASE', 'mean'].values * 100
single_product = customers_groups.loc['SINGLE_BUYER']['ONLINE_PURCHASE', 'mean'].values * 100
single_product = np.insert(single_product, 1, 0)

x = np.arange(len(labels))
width = 0.4

fig, ax = plt.subplots(figsize=(10,6))
rects1 = ax.bar(x - width/2, multi_product, width, label='Multi Buyer')
rects2 = ax.bar(x + width/2, single_product, width, label='Single Buyer')

ax.set_ylabel('Percentage of buyers %', fontsize = 'x-large')
ax.set_title('Percentage by Customer Group and Product Group', fontsize='x-large', weight = 'bold')
ax.set_xticks(x, labels)
ax.legend(fontsize = 'large')

ax.bar_label(rects1, padding=3, fmt='%.2f%%')
ax.bar_label(rects2, padding=3, fmt='%.2f%%')

fig.tight_layout()

plt.show()

In [ ]:
customers_groups.reset_index()

In [ ]:
totals = customers_groups.reset_index().groupby('PRODUCT_GROUP').sum()['ONLINE_PURCHASE', 'sum']

labels = totals.index.values

totals = totals.values

multi_product = 100 * customers_groups.loc['MULTI_BUYER', ('ONLINE_PURCHASE', 'sum')].values / totals

single_product = customers_groups.loc['SINGLE_BUYER', ('ONLINE_PURCHASE', 'sum')].values

single_product = np.insert(single_product, 1, 0).astype('i')

single_product =100 * single_product / totals

x = np.arange(len(labels))

width = 0.55

fig, ax = plt.subplots(figsize=(10,6))
rects1 = ax.bar(x, multi_product, width, label='Multi Buyer')
rects2 = ax.bar(x, single_product, width, bottom = multi_product, label='Single Buyer')

ax.set_ylabel('Percentage of buyers per Product', fontsize = 'x-large')
ax.set_title('Proportion of Customer Group Buyers by Product Group', fontsize='x-large', weight = 'bold')
ax.set_xticks(x, labels)
ax.legend(fontsize = 'large')

ax.bar_label(rects1, label_type='center', fmt='%.f%%', fontsize='large')
ax.bar_label(rects2, label_type='center', fmt='%.f%%', fontsize='large')

plt.show()

* Missing rows

In [ ]:
miss_azdias = (azdias.isna().sum(axis=0).sort_values(ascending=False)/azdias.shape[0])[:20]

In [ ]:
miss_customer = (customers.isna().sum(axis=0).sort_values(ascending=False)/customers.shape[0])[:20]

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15,13))

ax[0].bar(miss_azdias.index.values, height = miss_azdias)
ax[0].set_xticklabels(miss_azdias.index.values, rotation=30, ha='right')
ax[0].grid(True, ls = ':')


ax[1].bar(miss_customer.index.values, height = miss_customer)
ax[1].set_xticklabels(miss_customer.index.values, rotation=30, ha='right')
ax[1].grid(True, ls = ':')

fig.tight_layout()

plt.show()

* Missing columns

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(12,11))

ax[0].hist(azdias.isna().sum(axis=1) / azdias.shape[1], density=True)
ax[0].grid(True, ls = ':')
ax[0].set_title('Azdias', fontsize = 'x-large')
ax[0].set_xlabel('Percentage of empty columns', fontsize = 'large')
ax[0].set_ylabel('Density', fontsize = 'large')

ax[1].hist(customers.isna().sum(axis=1) / customers.shape[1], density = True)
ax[1].grid(True, ls = ':')
ax[1].set_title('Customers', fontsize = 'x-large')
ax[1].set_xlabel('Percentage of empty columns', fontsize = 'large')
ax[1].set_ylabel('Density', fontsize = 'large')

fig.suptitle('Missing columns', fontweight = 'bold', fontsize = 'xx-large')
fig.tight_layout()

plt.show()


In [ ]:
azdias_nuniques.describe()

In [ ]:
customers_nunique = customers.nunique()

In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(14,6))

ax[0].boxplot(x=[azdias_nuniques, customers_nunique])
ax[0].set_ylim(0, 100)
ax[0].set_title('Number of unique values per column', fontsize = 'x-large', fontweight='bold')
ax[0].set_xticklabels(labels=['Azdias', 'Customers'], fontsize = 'large')

ax[1].boxplot(x=[azdias_nuniques, customers_nunique])
ax[1].set_ylim(0, 15)
ax[1].set_title('Number of unique values per column', fontsize = 'x-large', fontweight='bold')
ax[1].set_xticklabels(labels=['Azdias', 'Customers'], fontsize = 'large')

plt.show()

In [ ]:
cat_col = azdias_nuniques[azdias_nuniques<100].index.values

In [ ]:
azdias.dtypes.value_counts()

In [ ]:
customers.dtypes.value_counts()

In [ ]:
distances = dict()
for col in cat_col:
    aux_azd = azdias[col].value_counts() / azdias.shape[0]
    aux_cust = customers[col].value_counts() / customers.shape[0]
    dfs = pd.merge(aux_azd.reset_index(), aux_cust.reset_index(), left_on='index', right_on= 'index')
    distances[col] = dfs.set_index('index').diff(axis=1).iloc[:,1].abs().sum()


In [ ]:
dfs

In [ ]:
distances = pd.Series(distances).sort_values(ascending=False)
distances

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))

ax.bar(x = distances.index.values[:20], height = distances.values[:20])
ax.grid(True, ls = ':')
ax.set_xticklabels(distances.index.values[:20], rotation=30, ha='right')

fig.suptitle('Distance between azdias and Customers', fontweight = 'bold', fontsize = 'xx-large')
fig.tight_layout()

plt.show()

In [ ]:
distances.describe()

In [ ]:
azdias[azdias_nuniques[azdias_nuniques>100].index.values].describe()

In [ ]:
customers[azdias_nuniques[azdias_nuniques > 100].index.values].describe()

In [ ]:
corr = azdias.corr()

In [ ]:
corr.reset_index()

In [ ]:
corr_list = corr.reset_index().melt(id_vars = 'index')

In [ ]:
corr_list.query('index != variable').describe()

In [ ]:
fig, ax = plt.subplots(figsize=(11, 6))

ax.hist(x=corr_list.query('index != variable').value, bins = 10, density = True)
ax.grid(True, ls = ':')

fig.suptitle("Pearson's Correlation Density Histogram", fontweight = 'bold', fontsize = 'xx-large')

plt.show()

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

One hot Enconding for the string variables

In [ ]:
azdias.select_dtypes('object').columns.values

In [ ]:
estimators = [('col', ColumnTransformer([('one_hot', OneHotEncoder(), azdias.select_dtypes('object').columns.values)], 
                                        remainder = 'passthrough'))]

In [ ]:
pipe = Pipeline(estimators)

In [ ]:
pipe.fit_transform(X = azdias)

In [ ]:
azdias.select_dtypes(include='object')

In [ ]:
azdias.select_dtypes(include=['object', 'string']).columns.values

In [ ]:
enc = OneHotEncoder(handle_unknown = 'ignore')

In [ ]:
df = enc.fit_transform(azdias.select_dtypes(include=['object']))

In [ ]:
enc.get_feature_names_out()

In [ ]:
azdias[['CAMEO_DEU_2015', 'CAMEO_DEUG_2015', 'CAMEO_INTL_2015', 'D19_LETZTER_KAUF_BRANCHE', 'OST_WEST_KZ']]

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which includes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [ ]:
mailout_train = pd.read_csv('data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter.

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')